In [1]:
import keras
from keras import layers
import keras_tuner as kt
import mlflow
import mlflow.keras
from emotion_model_pipeline import load_data
import tensorflow as tf
import mlflow

In [2]:
# Load the data
data = load_data(
    X_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/X.npy',  # Path to the features file
    y_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/y.npy',  # Path to the labels file
)

# Print the data shapes
print(f"Training data shape: {data['X_train'].shape}")
print(f"Testing data shape: {data['X_test'].shape}")

Training data shape: (20973, 48, 48, 1)
Testing data shape: (5244, 48, 48, 1)


In [8]:
# Define your experiment name
experiment_name = "emotion_recognition_experiment_2"

# Set the experiment
mlflow.set_experiment(experiment_name)

# Verify the experiment was created (you can print the current experiment to check)
experiment = mlflow.get_experiment_by_name(experiment_name)
print(experiment)


2024/11/15 12:00:05 INFO mlflow.tracking.fluent: Experiment with name 'emotion_recognition_experiment_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Surface/NU/Vibify/pipelines/dags/src/mlruns/545330959567176815', creation_time=1731690005934, experiment_id='545330959567176815', last_update_time=1731690005934, lifecycle_stage='active', name='emotion_recognition_experiment_2', tags={}>


In [6]:
# Define the model-building function with hyperparameter tuning
def build_model(hp):
    model = keras.Sequential()
    
    # Choose number of convolutional layers
    model.add(layers.InputLayer(input_shape=(48, 48, 1)))
    
    for i in range(hp.Int('conv_layers', 1, 3)):  # Randomly choose number of conv layers (1 to 3)
        model.add(layers.Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32),  # Random filter size
            kernel_size=(3, 3),
            activation='relu'
        ))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Flatten())
    
    # Add a fully connected layer with number of units
    model.add(layers.Dense(
        hp.Int('units', min_value=64, max_value=512, step=64),  # Random number of units
        activation='relu'
    ))

    # Output layer with 4 classes (emotion classification)
    model.add(layers.Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
